In [1]:
""" @namespace Demos of solo12 step adjustment
@file
@copyright Copyright (c) 2017-2021,
           New York University and Max Planck Gesellschaft,
           License BSD-3-Clause
@example
"""
import numpy as np
import pybullet as p
from robot_properties_solo.config import Solo12Config
from robot_properties_solo.solo12wrapper import Solo12Robot
from mim_control.robot_centroidal_controller import RobotCentroidalController
from mim_control.robot_impedance_controller import RobotImpedanceController
from py_reactive_planners.lipm_simulator import LipmSimpulator
from reactive_planners import DcmReactiveStepper
import pinocchio as pin
from scipy.spatial.transform import Rotation
from bullet_utils.env import BulletEnvWithGround

np.set_printoptions(suppress=True, precision=2)
pin.switchToNumpyArray()

def zero_cnt_gain(kp, cnt_array):
    gain = np.array(kp).copy()
    for i, v in enumerate(cnt_array):
        if v == 1:
            gain[3 * i : 3 * (i + 1)] = 0.0
    return gain


def yaw(q):
    return np.array(
        Rotation.from_quat([np.array(q)[3:7]]).as_euler("xyz", degrees=False)
    )[0, 2]

In [2]:
# Create a robot instance. This initializes the simulator as well.
env = BulletEnvWithGround()
robot = env.add_robot(Solo12Robot)
tau = np.zeros(12)

time = 0
sim_freq = 1000  # Hz
ctrl_freq = 1000
plan_freq = 1000

p.resetDebugVisualizerCamera(1.6, 50, -35, (0.0, 0.0, 0.0))
p.setTimeStep(1.0 / sim_freq)
p.setRealTimeSimulation(0)
for ji in range(12):
    p.changeDynamics(
        robot.robotId,
        ji,
        linearDamping=0.04,
        angularDamping=0.04,
        restitution=0.0,
        lateralFriction=4.0,
        spinningFriction=5.6,
    )

In [3]:
q = np.array(Solo12Config.initial_configuration)
q[0] = 0.
q[3:7] = pin.Quaternion(pin.rpy.rpyToMatrix(0., 0., 0.)).coeffs() # 
qdot = np.matrix(Solo12Config.initial_velocity).T
robot.reset_state(q, qdot)
total_mass = sum([i.mass for i in robot.pin_robot.model.inertias[1:]])
warmup = 5
kp = np.array(12 * [150.0])
kd = 12 * [5.0]
robot_config = Solo12Config()
config_file = robot_config.paths["imp_ctrl_yaml"]
solo_leg_ctrl = RobotImpedanceController(robot, config_file)
centr_controller = RobotCentroidalController(
    robot_config,
    mu=0.6,
    kc=[100, 100, 200],
    dc=[10, 10, 10],
    kb=[25, 25, 25.],
    db=[22.5, 22.5, 22.5],
    qp_penalty_lin=[1e0, 1e0, 1e6],
    qp_penalty_ang=[1e6, 1e6, 1e6],
)
is_left_leg_in_contact = True
l_min = -0.1
l_max = 0.1
w_min = -0.08
w_max = 0.2
t_min = 0.1
t_max = 1.0
l_p = 0.00  # Pelvis width
com_height = 0.25
weight = [1, 1, 5, 1000, 1000, 100000, 100000, 100000, 100000]
mid_air_foot_height = 0.05
control_period = 0.001
planner_loop = 0.010
x_des_local = [
    q[0].item(),
    q[1].item() + 0.,
    0.0,
    q[0].item(),
    q[1].item() - 0.,
    0.0,
]
past_x = x_des_local.copy()

v_des = np.array([0.0, 0.0, 0.0])
y_des = 0.1 # Speed of the yaw angle

dcm_reactive_stepper = DcmReactiveStepper()
dcm_reactive_stepper.initialize(
    is_left_leg_in_contact,
    l_min,
    l_max,
    w_min,
    w_max,
    t_min,
    t_max,
    l_p,
    com_height,
    weight,
    mid_air_foot_height,
    control_period,
    planner_loop,
    x_des_local[:3],
    x_des_local[3:],
    v_des,
)

dcm_reactive_stepper.set_desired_com_velocity(v_des)

x_com = [[0.0], [0.0], [com_height]]
com_des = np.array([0., 0.0])
yaw_des = yaw(q)
cnt_array = [1, 1]
time = 0
control_time = 0
open_loop = True
dcm_force = np.array([0.0, 0.0, 0.0])
offset = 0.015  # foot radius
dcm_reactive_stepper.start()

for i in range(1000+ warmup):
#     if i == 1000:
#         v_des = np.array([0.3, 0.3, 0.])
#         
#     if i == 4000:
#         p.applyExternalForce(robot.robotId, -1, [-200., 200., 0.], [0., 0., 0.], p.LINK_FRAME)
    
    last_qdot = qdot
    q, qdot = robot.get_state()
    robot.pin_robot.com(q, qdot)
    robot.update_pinocchio(q, qdot)
    x_com = robot.pin_robot.com(q, qdot)[0]
    xd_com = robot.pin_robot.com(q, qdot)[1]
    
#     com_des += v_des[:2] * 0.001
#     yaw_des += y_des * 0.001


    # TODO: Handle rotations here.
#         R = pin.rpy.rpyToMatrix(0., 0., yaw(q))
        
    omega = 0.1
    yaw_des = np.sin(omega * i * 0.001)
    y_des = omega * np.cos(omega * i * 0.001)
    
    w_com = centr_controller.compute_com_wrench(
        q.copy(),
        qdot.copy(),
        [com_des[0], com_des[1], com_height],
        v_des,
        pin.Quaternion(pin.rpy.rpyToMatrix(0., 0., yaw_des)).coeffs(),
        [0.0, 0.0, y_des],
    )
    
    x_des_local = np.hstack([
        np.array([ 0.195,  0.147, offset]),
        np.array([ 0.195, -0.147, offset]),
        np.array([-0.195,  0.147, offset]),
        np.array([-0.195, -0.147, offset])
    ])
    des_vel = np.zeros(12)
    
    cnt_array = [1, 1, 1, 1]
    F = centr_controller.compute_force_qp(q, qdot, cnt_array, w_com)
    
    tau = solo_leg_ctrl.return_joint_torques(
        q.copy(),
        qdot.copy(),
        np.zeros(12),
        np.zeros(12),
#         zero_cnt_gain(kp, cnt_array),
#         zero_cnt_gain(kd, cnt_array),
        x_des_local,
        des_vel,
        F,
    )
    control_time += 0.001

    robot.send_joint_command(tau)
    p.stepSimulation()

dcm_reactive_stepper.stop()